In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
from make_diagnostic_plots import *
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.font_manager import FontProperties


import matplotlib 

matplotlib.rc('xtick', labelsize=36) 
matplotlib.rc('ytick', labelsize=36) 

# use latex for font rendering
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)

matplotlib.rcParams.update({'errorbar.capsize': 0})




In [ ]:
path_to_dir = './phoebe_results/'
planet1 = 'Earth'
planet2 = 'Saturn'
nruns = 7

## let's make the diagnostic plots

#### this will take some time, so we can skip it also....

In [ ]:
for ii in range(1, nruns+1):
    for jj in range(1, 11):
        run_directory = path_to_dir+'run'+str(ii)+'.'+str(jj)
        make_diagnostic_plots(run_directory+'/sim_data.csv', run_directory+'/posterior.cdf', run_directory+'/planet_params.csv', 
                              2, run_directory+'/diagnostic', run_directory+'/corner', run_directory+'/posterior_stats',
                              run_directory+'/posterior', noRoman=False)



In [ ]:
def load_posterior_params(filename, nplanets):
    trace = arviz.from_netcdf(filename)


    parameters = ['P', 'ecc', 'tperi', 'omega', 'Omega', 'incl', 'm_planet']


    params_earth = defaultdict(list)
    params_jup = defaultdict(list)
    params_earth_err = defaultdict(list)
    params_jup_err = defaultdict(list)

    table_params_all = []
    posterior_meds_all = []
    posterior_errs_all = []
    for ii in range(0, nplanets):
        table_params = defaultdict(list)
        posterior_meds = defaultdict(list)
        posterior_errs = defaultdict(list)
        for param in parameters:


            planet_med = np.median(trace.posterior[param].values[:, :, ii])

            planet_quantile = [np.quantile(trace.posterior[param].values[:, :, ii], 0.16),
                                np.quantile(trace.posterior[param].values[:, :, ii], 0.84)]



            planet_err = [
                planet_med - planet_quantile[0],
                planet_quantile[1] - planet_med 
            ]



            #print(param + "_" + str(ii+1) + ": " + str(planet_med) + " +/- " + str(planet_err))

            posterior_meds[param] = np.round(planet_med, 3)
            posterior_errs[param] = np.round(planet_err, 3)
            table_params[param].append(str(np.round(planet_med, 3)) + " (+" + 
                                             str(np.round(planet_err[1], 3)) + " -" + 
                                             str(np.round(planet_err[0], 3)) + ")")






        table_params_all.append(table_params)
        posterior_meds_all.append(posterior_meds)
        posterior_errs_all.append(posterior_errs)
        #print("")
        #print("")



    return table_params_all, posterior_meds_all, posterior_errs_all


In [ ]:
def load_input_params_and_keys(filename):
    planet_param_keys = ['period', 'ecc', r'T$_\mathrm{per}$', r'$\omega$', r'$\Omega$', 'inc', 'mass']
    planet_params_df = pd.read_csv(filename)
    
    planet_params = []
    for row in planet_params_df.iterrows():
        params = []
        params.append(row[1]['period'])
        params.append(row[1]['ecc'])
        params.append(row[1]['Tper'])
        params.append(row[1]['omega'])
        params.append(row[1]['Omega'])
        params.append(row[1]['inclination'])
        params.append(row[1]['mass']*332900.)


        planet_params.append(params)
    
    return planet_param_keys, planet_params

In [ ]:
import pandas as pd
from scipy.stats import median_abs_deviation
import os


df = {}

masses_planet1 = []
masses_err_planet1 = []
masses_err_planet1_std = []
masses_planet2 = []
masses_err_planet2 = []
masses_err_planet2_std = []

periods_planet1 = []
periods_err_planet1 = []
periods_err_planet1_std = []
periods_planet2 = []
periods_err_planet2 = []
periods_err_planet2_std = []

eccs_planet1 = []
eccs_err_planet1 = []
eccs_err_planet1_std = []
eccs_planet2 = []
eccs_err_planet2 = []
eccs_err_planet2_std = []


roman_errs = []
roman_durations = []
for ii in range(1, nruns+1):
    masses_planet1_sub = []
    masses_err_planet1_sub = []
    masses_planet2_sub = []
    masses_err_planet2_sub = []

    periods_planet1_sub = []
    periods_err_planet1_sub = []
    periods_planet2_sub = []
    periods_err_planet2_sub = []

    eccs_planet1_sub = []
    eccs_err_planet1_sub = []
    eccs_planet2_sub = []
    eccs_err_planet2_sub = []


    roman_errs_sub = []
    roman_durations_sub = []
    for jj in range(1, 11):

        run_directory = path_to_dir+'run'+str(ii)+'.'+str(jj)
        
        max_rhat = 1.05
        if rhat_passes_test(run_directory+'/posterior.cdf', max_rhat = max_rhat):

            roman_err = float(pd.read_csv(run_directory+'/observe_params.csv')['roman_err'])
            roman_errs_sub.append(roman_err)

            roman_duration = float(pd.read_csv(run_directory+'/observe_params.csv')['roman_duration'])
            roman_durations_sub.append(roman_duration)

            posteriors = load_posterior_params(run_directory+'/posterior.cdf', 2)


            medians = posteriors[1]
            errs = posteriors[2]



            mass_planet1 = medians[0]['m_planet']
            mass_planet2 = medians[1]['m_planet']
            mass_err_planet1 = errs[0]['m_planet']
            mass_err_planet2 = errs[1]['m_planet']

            masses_planet1_sub.append(mass_planet1)
            masses_planet2_sub.append(mass_planet2)
            masses_err_planet1_sub.append(mass_err_planet1)
            masses_err_planet2_sub.append(mass_err_planet2)






            period_planet1 = medians[0]['P']
            period_planet2 = medians[1]['P']
            period_err_planet1 = errs[0]['P']
            period_err_planet2 = errs[1]['P']

            periods_planet1_sub.append(period_planet1)
            periods_planet2_sub.append(period_planet2)
            periods_err_planet1_sub.append(period_err_planet1)
            periods_err_planet2_sub.append(period_err_planet2)




            ecc_planet1 = medians[0]['ecc']
            ecc_planet2 = medians[1]['ecc']
            ecc_err_planet1 = errs[0]['ecc']
            ecc_err_planet2 = errs[1]['ecc']

            eccs_planet1_sub.append(ecc_planet1)
            eccs_planet2_sub.append(ecc_planet2)
            eccs_err_planet1_sub.append(ecc_err_planet1)
            eccs_err_planet2_sub.append(ecc_err_planet2)
            
        else:
            print(run_directory + ' rhat > + ' + str(max_rhat))


        
        
        
        
    med_masses_planet1 = np.median(np.array(masses_planet1_sub))
    med_masses_error_planet1 = np.median(np.array(masses_err_planet1_sub).T, axis=1)
    std_masses_error_planet1 = median_abs_deviation(np.array(masses_err_planet1_sub).T, axis=1)
    med_masses_planet2 = np.median(np.array(masses_planet2_sub))
    med_masses_error_planet2 = np.median(np.array(masses_err_planet2_sub).T, axis=1)
    std_masses_error_planet2 = median_abs_deviation(np.array(masses_err_planet2_sub).T, axis=1)

    masses_planet1.append(med_masses_planet1)
    masses_err_planet1.append(med_masses_error_planet1)
    masses_err_planet1_std.append(std_masses_error_planet1)
    masses_planet2.append(med_masses_planet2)
    masses_err_planet2.append(med_masses_error_planet2)
    masses_err_planet2_std.append(std_masses_error_planet2)
    
    
    
    med_periods_planet1 = np.median(np.array(periods_planet1_sub))
    med_periods_error_planet1 = np.median(np.array(periods_err_planet1_sub).T, axis=1)
    std_periods_error_planet1 = median_abs_deviation(np.array(periods_err_planet1_sub).T, axis=1)
    med_periods_planet2 = np.median(np.array(periods_planet2_sub))
    med_periods_error_planet2 = np.median(np.array(periods_err_planet2_sub).T, axis=1)
    std_periods_error_planet2 = median_abs_deviation(np.array(periods_err_planet2_sub).T, axis=1)

    periods_planet1.append(med_periods_planet1)
    periods_err_planet1.append(med_periods_error_planet1)
    periods_err_planet1_std.append(std_periods_error_planet1)
    periods_planet2.append(med_periods_planet2)
    periods_err_planet2.append(med_periods_error_planet2)
    periods_err_planet2_std.append(std_periods_error_planet2)

    
    
    
    med_eccs_planet1 = np.median(np.array(eccs_planet1_sub))
    med_eccs_error_planet1 = np.median(np.array(eccs_err_planet1_sub).T, axis=1)
    std_eccs_error_planet1 = median_abs_deviation(np.array(eccs_err_planet1_sub).T, axis=1)
    med_eccs_planet2 = np.median(np.array(eccs_planet2_sub))
    med_eccs_error_planet2 = np.median(np.array(eccs_err_planet2_sub).T, axis=1)
    std_eccs_error_planet2 = median_abs_deviation(np.array(eccs_err_planet2_sub).T, axis=1)


    eccs_planet1.append(med_eccs_planet1)
    eccs_err_planet1.append(med_eccs_error_planet1)
    eccs_err_planet1_std.append(std_eccs_error_planet1)
    eccs_planet2.append(med_eccs_planet2)
    eccs_err_planet2.append(med_eccs_error_planet2)
    eccs_err_planet2_std.append(std_eccs_error_planet2)


    roman_errs.append(roman_err)
    roman_durations.append(roman_duration)
    
    
    

    
    
masses_err_planet1 = np.array(masses_err_planet1).T
masses_err_planet1_std = np.array(masses_err_planet1_std).T
masses_err_planet2 = np.array(masses_err_planet2).T
masses_err_planet2_std = np.array(masses_err_planet2_std).T

periods_err_planet1 = np.array(periods_err_planet1).T
periods_err_planet1_std = np.array(periods_err_planet1_std).T
periods_err_planet2 = np.array(periods_err_planet2).T
periods_err_planet2_std = np.array(periods_err_planet2_std).T

eccs_err_planet1 = np.array(eccs_err_planet1).T
eccs_err_planet1_std = np.array(eccs_err_planet1_std).T
eccs_err_planet2 = np.array(eccs_err_planet2).T
eccs_err_planet2_std = np.array(eccs_err_planet2_std).T



    

















In [ ]:
planet_params = load_input_params_and_keys(path_to_dir+'run1.1'+'/planet_params.csv')

mass_truth_planet1 = planet_params[1][0][6]
mass_truth_planet2 = planet_params[1][1][6]

period_truth_planet1 = planet_params[1][0][0]
period_truth_planet2 = planet_params[1][1][0]

ecc_truth_planet1 = planet_params[1][0][1]
ecc_truth_planet2 = planet_params[1][1][1]






In [ ]:
def plot_medians(param_medians, param_errs, param_truths, 
                 param_name, param_unit, planet1, planet2, nruns, roman_errs, colors):
    
    
    ys = np.arange(1, nruns+1, 1)

    labels = []
    for ii in range(0, nruns):
        if 'nan' not in str(roman_errs[ii]):
            label = str(int(roman_errs[ii]*1e6)) + r' $\mu$as and ' + str(int(roman_durations[ii])) + ' yrs'
        else:
            label = 'no Roman obs'
        labels.append(label)
    
    
    
    
    fig,ax = plt.subplots(1,2, figsize=[23,9], sharey = True)

    for ii in range(0, nruns):
        if ii % 2 != 0:
            plot_color1 = colors[0][0]
            plot_color2 = colors[1][0]
            
        else:
            plot_color1 = colors[0][1]
            plot_color2 = colors[1][1]
        ax[1].errorbar(param_medians[0][ii]/param_truths[0], ys[ii], xerr = np.array([param_errs[0].T[ii]]).T/param_truths[0], 
                       ls='', marker='o', ms=13, color = plot_color1, elinewidth=3)
        ax[0].errorbar(param_medians[1][ii]/param_truths[1], ys[ii], xerr = np.array([param_errs[1].T[ii]]).T/param_truths[1], 
                       ls='', marker='o', ms=13, color = plot_color2, elinewidth=3)

    ax[1].axvline(1, 0, 1, color = 'k', lw = 3, ls = '--', label = 'truth')
    ax[0].axvline(1, 0, 1, color = 'k', lw = 3, ls = '--', label = 'truth')
    
    if param_unit == '':
        ax[1].set_xlabel(planet1 + ' ' + param_name + ' Recovered', fontsize = 36)
        ax[0].set_xlabel(planet2 + ' ' + param_name + ' Recovered', fontsize = 36)
        
    else:
        ax[1].set_xlabel(planet1 + ' ' + param_name + ' Recovered ' + 
                         '[' + param_unit + r'$_\mathrm{' + planet1 + '}$]', fontsize = 36)
        ax[0].set_xlabel(planet2 + ' ' + param_name + ' Recovered ' + 
                         '[' + param_unit + r'$_\mathrm{' + planet2 + '}$]', fontsize = 36)



    ax[0].set_yticks(ys)
    ax[0].set_yticklabels(labels, fontsize = 36)

    if param_name=='Mass':
        fig.suptitle(r'Recovered ' + param_name + 'es vs. Roman Observing Plan', x=0.57, fontsize=69)
    else:
        fig.suptitle(r'Recovered ' + param_name + 's vs. Roman Observing Plan', x=0.57, fontsize=69)
        
    ax[1].legend(fontsize = 36)
    ax[0].legend(fontsize = 36)
    
    ax[1].set_ylim(np.min(ys)-.5, np.max(ys)+1.5)
    ax[0].set_ylim(np.min(ys)-.5, np.max(ys)+1.5)
    
    ax[0].tick_params(axis='both', which='major', direction = 'inout', length = 13, width = 3)
    ax[0].tick_params(axis='both', which='minor', direction = 'inout', length = 13, width = 3)
    
    ax[1].tick_params(axis='x', which='major', direction = 'inout', length = 13, width = 3)
    ax[1].tick_params(axis='x', which='minor', direction = 'inout', length = 13, width = 3)
    
    ax[0].grid(True, alpha=0.36) 
    ax[1].grid(True, alpha=0.36) 



    
    
    fig.tight_layout()


    fig.savefig(planet1+'_'+planet2+'_' + param_name + '.pdf')


In [ ]:
def plot_uncertainties(posterior_err_params, mad_posterior_err_params, param_truths, 
                       colors, param_name, param_unit, planet1, planet2):
    '''
    posterior_params = list of the parameters to plot, of length 2, 
    with sub-arrays that contain the parameter information at various precision assumptions
    
    colors = list of 2 lists of 2 colors
    '''
    mean_param_err_planet1, mean_param_err_planet2 = posterior_err_params
    mad_param_err_planet1, mad_param_err_planet2 = mad_posterior_err_params
    
    mad_param_err_planet1 = mad_param_err_planet1.T
    mad_param_err_planet2 = mad_param_err_planet2.T
    
    print(param_name)
    fig,ax = plt.subplots(1,2, figsize=[23,9])

    xs = np.zeros(np.shape(roman_errs))
    earth_vals = []
    cgg_vals = []
    for ii in range(0, nruns):
        if roman_errs[ii] == 5e-06:
            xs[ii] = 1

        elif roman_errs[ii] == 1e-05:
            xs[ii] = 2

        elif roman_errs[ii] == 2e-05:
            xs[ii] = 3

        elif np.isnan(roman_errs[ii]):
            xs[ii] = 4

        
        
        if roman_durations[ii] == 5.0:
            ax[1].errorbar(xs[ii]-.1, mean_param_err_planet1[ii]/param_truths[0], 
                           yerr=([mad_param_err_planet1[ii][0]], [mad_param_err_planet1[ii][1]])/param_truths[0],
                           marker = 'o', color = colors[0][0], ms = 13, elinewidth=3, zorder = -10000)
            ax[0].errorbar(xs[ii]-.1, mean_param_err_planet2[ii]/param_truths[1], 
                           yerr=([mad_param_err_planet2[ii][0]], [mad_param_err_planet2[ii][1]])/param_truths[1],
                           marker = 'o', color = colors[1][0], ms = 13, elinewidth=3, zorder = -10000)
        elif roman_durations[ii] == 10.0:
            ax[1].errorbar(xs[ii]+.1, mean_param_err_planet1[ii]/param_truths[0], 
                           yerr=([mad_param_err_planet1[ii][0]], [mad_param_err_planet1[ii][1]])/param_truths[0],
                           marker = 'o', color = colors[0][1], ms = 13, elinewidth=3, zorder = 10000)
            ax[0].errorbar(xs[ii]+.1, mean_param_err_planet2[ii]/param_truths[1], 
                           yerr=([mad_param_err_planet2[ii][0]], [mad_param_err_planet2[ii][1]])/param_truths[1],
                           marker = 'o', color = colors[1][1], ms = 13, elinewidth=3, zorder = 10000)
            
            earth_vals.append(np.round(mean_param_err_planet1[ii]/param_truths[0], 4))
            cgg_vals.append(np.round(mean_param_err_planet2[ii]/param_truths[1], 4))
            
        
    best_earth_val = np.min(earth_vals)
    worst_earth_val = np.max(earth_vals)

    best_cgg_val = np.min(cgg_vals)
    worst_cgg_val = np.max(cgg_vals)

    ratio_earth_val = worst_earth_val/best_earth_val
    ratio_cgg_val = worst_cgg_val/best_cgg_val

    print('earth', best_earth_val, worst_earth_val, ratio_earth_val)
    print('cgg', best_cgg_val, worst_cgg_val, ratio_cgg_val)
    print('')
            


    ax[1].set_xlabel(r'Roman Precision [$\mu$as]', fontsize = 36)
    ax[0].set_xlabel(r'Roman Precision [$\mu$as]', fontsize = 36)

    
    ax[1].set_ylabel('Fractional Uncertainty ' + 
                     '[' + param_unit + r'$_\mathrm{' + planet1 + '}$]', fontsize = 36)
    ax[0].set_ylabel('Fractional Uncertainty ' + 
                     '[' + param_unit + r'$_\mathrm{' + planet2 + '}$]', fontsize = 36)

    ax[1].set_xticks([1,2,3,4])
    ax[1].set_xticklabels([5, 10, 20, 'No Roman'])

    ax[0].set_xticks([1,2,3,4])
    ax[0].set_xticklabels([5, 10, 20, 'No Roman'])


    ax[1].text(0.05, 0.9, planet1, horizontalalignment='left',
         verticalalignment='center', transform=ax[1].transAxes, fontsize=36)
    ax[0].text(0.05, 0.9, planet2, horizontalalignment='left',
         verticalalignment='center', transform=ax[0].transAxes, fontsize=36)


    ax[1].plot(0.05, 0.8, 'o', ms = 18, color = colors[0][0], transform=ax[1].transAxes)
    ax[1].text(0.09, 0.8, '5 yr Roman', horizontalalignment='left',
               verticalalignment='center', transform=ax[1].transAxes, fontsize=36)
    ax[1].plot(0.05, 0.7, 'o', ms = 18, color = colors[0][1], transform=ax[1].transAxes)
    ax[1].text(0.09, 0.7, '10 yr Roman', horizontalalignment='left',
               verticalalignment='center', transform=ax[1].transAxes, fontsize=36)

    ax[0].plot(0.05, 0.8, 'o', ms = 18, color = colors[1][0], transform=ax[0].transAxes)
    ax[0].text(0.09, 0.8, '5 yr Roman', horizontalalignment='left',
               verticalalignment='center', transform=ax[0].transAxes, fontsize=36)

    ax[0].plot(0.05, 0.7, 'o', ms = 18, color = colors[1][1], transform=ax[0].transAxes)
    ax[0].text(0.09, 0.7, '10 yr Roman', horizontalalignment='left',
               verticalalignment='center', transform=ax[0].transAxes, fontsize=36)
    
    
    
    ax[0].tick_params(axis='both', which='major', direction = 'inout', length = 13, width = 3)
    ax[0].tick_params(axis='both', which='minor', direction = 'inout', length = 13, width = 3)
    
    ax[1].tick_params(axis='both', which='major', direction = 'inout', length = 13, width = 3)
    ax[1].tick_params(axis='both', which='minor', direction = 'inout', length = 13, width = 3)
    
    ax[0].grid(True, alpha=0.36) 
    ax[1].grid(True, alpha=0.36)


    rect = patches.Rectangle((0.02, 0.65), .36, .3, linewidth=1, edgecolor='grey', facecolor='grey',
                            transform=ax[1].transAxes, alpha=.6, joinstyle='round')
    ax[1].add_patch(rect)

    rect = patches.Rectangle((0.02, 0.65), .36, .3, linewidth=1, edgecolor='grey', facecolor='grey',
                            transform=ax[0].transAxes, alpha=.6)
    ax[0].add_patch(rect)

    ax[1].set_ylim(np.min(mean_param_err_planet1/param_truths[0])/1.5, np.max(mean_param_err_planet1/param_truths[0])*1.8)

    ax[0].set_ylim(np.min(mean_param_err_planet2/param_truths[1])/1.5, np.max(mean_param_err_planet2/param_truths[1])*1.5)

    fig.suptitle('Fractional ' + param_name + ' Uncertainty vs. Roman Observing Plan', fontsize=69)
    fig.tight_layout()

    fig.savefig(planet1 + '_' + planet2 + '_' + param_name + '_errs.pdf')

In [ ]:
def make_median_table(param_medians, param_errs, param_truths, 
                      param_name, param_unit, planet1, planet2, nruns, roman_errs):

    rowlabels = []
    for ii in range(0, nruns):
        if 'nan' not in str(roman_errs[ii]):
            label = str(int(roman_errs[ii]*1e6)) + r' $\mu$as and ' + str(int(roman_durations[ii])) + ' years of Roman'
        else:
            label = 'no Roman observations'
        rowlabels.append(label)


    param_unit_table1 = ' [' + param_unit + r'$_\mathrm{' + planet1 + '}$]'
    param_unit_table2 = ' [' + param_unit + r'$_\mathrm{' + planet2 + '}$]'
    col_labels = ['Roman Observing Plan',
                  planet1 + r' median +/- 1$\sigma$ ' + param_name + param_unit_table1, 
                  planet2 + r' median +/- 1$\sigma$ ' + param_name + param_unit_table2]


    fig, ax = plt.subplots(figsize=[9,7])

    vals = [rowlabels]
    for ii in range(0, len(param_medians)):
        vals.append([])

        param_med = param_medians[ii]
        param_err = param_errs[ii]
        truth = param_truths[ii]
        for jj in range(0, len(param_med)):
            val = str(np.round(param_med[jj]/truth,4)) + ' (+' + \
            str(np.round(param_err[1][jj]/truth, 4)) + ' -' + \
            str(np.round(param_err[0][jj]/truth, 4)) + ')'

            vals[ii+1].append(val)



    vals = np.array(vals).T

    # hide axes
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')

    table=ax.table(cellText=vals, colLabels=col_labels, loc='center')
    table.set_fontsize(72)
    table.scale(3,6)





    fig.tight_layout()

    fig.savefig(planet1+'_'+planet2+'_' + param_name + '_table.pdf', bbox_inches='tight')



In [ ]:

reds = ['#a9002e', '#c37c7a']
blues = ['#2b538c', '#3eb4ff']
greens = ['#00692a', '#6cde73']

colors = [greens, reds] 


plot_medians([masses_planet1, masses_planet2], [masses_err_planet1, masses_err_planet2], 
             [mass_truth_planet1, mass_truth_planet2],
             'Mass', 'M', 'Earth', 'Saturn', nruns, roman_errs, colors)


plot_medians([periods_planet1, periods_planet2], [periods_err_planet1, periods_err_planet2], 
             [period_truth_planet1, period_truth_planet2],
             'Period', 'P', 'Earth', 'Saturn', nruns, roman_errs, colors)




make_median_table([masses_planet1, masses_planet2], [masses_err_planet1, masses_err_planet2], 
                  [mass_truth_planet1, mass_truth_planet2],
                  'Mass', 'M', 'Earth', 'Saturn', nruns, roman_errs)


make_median_table([periods_planet1, periods_planet2], [periods_err_planet1, periods_err_planet2], 
                  [period_truth_planet1, period_truth_planet2],
                  'Period', 'P', 'Earth', 'Saturn', nruns, roman_errs)

In [ ]:
mean_periods_errs_planet1 = np.mean(periods_err_planet1.T, axis=1)
mean_periods_errs_planet2 = np.mean(periods_err_planet2.T, axis=1)

period_post = [mean_periods_errs_planet1, mean_periods_errs_planet2]
mad_period_post = [periods_err_planet1_std, periods_err_planet2_std]




mean_masses_err_planet1 = np.mean(masses_err_planet1.T, axis=1)
mean_masses_err_planet2 = np.mean(masses_err_planet2.T, axis=1)

mass_post = [mean_masses_err_planet1, mean_masses_err_planet2]
mad_mass_post = [masses_err_planet1_std, masses_err_planet2_std]




mean_eccs_err_planet1 = np.mean(eccs_err_planet1.T, axis=1)
mean_eccs_err_planet2 = np.mean(eccs_err_planet2.T, axis=1)


ecc_post = [mean_eccs_err_planet1, mean_eccs_err_planet2]
mad_ecc_post = [eccs_err_planet1_std, eccs_err_planet2_std]


reds = ['#a9002e', '#c37c7a']
blues = ['#2b538c', '#3eb4ff']
greens = ['#00692a', '#6cde73']

colors = [greens, reds]
#r'M$_\mathrm{Earth}$'

plot_uncertainties(mass_post, mad_mass_post, [mass_truth_planet1, mass_truth_planet2],
                   colors, 'Mass', 'M', 'Earth', 'Saturn')


plot_uncertainties(period_post, mad_period_post, [period_truth_planet1, period_truth_planet2],
                   colors, 'Period', 'P', 'Earth', 'Saturn')

